In [35]:
from crewai import Agent, Task, Crew
from crewai_tools import SerperDevTool
from dotenv import load_dotenv
import os
import datetime

In [36]:
# Load environment variables
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["SERPER_API_KEY"] = os.getenv("SERPER_API_KEY")

In [37]:
# Initialize LLM and Tool
llm = "gpt-4-0125-preview"
search_tool = SerperDevTool()

companies = input("Enter companies to analyze (comma-separated, e.g., Apple, Tesla): ").split(", ")
companies_str = ", ".join(companies)

# Get the current date and time
current_time = datetime.datetime.now()
filename_datetime = current_time.strftime("%Y/%m/%d/%H")

In [38]:
# Define Agents with company-specific focus and search tool
research_agent = Agent(
    role='Company Researcher',
    goal=f'Gather financial data and market position for {companies_str} up to the latest available date and time, specifically {current_time}.',
    backstory='You are an expert in company-specific financial research with access to latest available date and time web data.',
    verbose=True,
    llm=llm,
    tools=[search_tool]
)

analysis_agent = Agent(
    role='Company Analyst',
    goal=f'Analyze investment potential for {companies_str} based on the latest data available up to {current_time}',
    backstory='Your name is jack and you are a CFA holder that specializes in evaluating company investment opportunities.',
    verbose=True,
    llm=llm,
    tools=[search_tool]
)

risk_analysis_agent = Agent(
    role='Risk Analyst',
    goal=f'Analyze risks assessment for {companies_str} based on the latest data available up to {current_time}',
    backstory='Your name is ethan and you are a FRM holder that specializes in evaluating company investment risks.',
    verbose=True,
    llm=llm,
    tools=[search_tool]
)

report_agent = Agent(
    role='Investment Advisor',
    goal=f'Create investment recommendations for {companies_str} with current insights up to {current_time}.',
    backstory='You are a financial advisor creating tailored investment reports and include all the ratios provided by the analysis agent.',
    verbose=True,
    llm=llm
)

In [ ]:
# Define Tasks
research_task = Task(
    description=f'Research current financial performance, market trends, and news for {companies_str} up to {current_time} using web search.',
    agent=research_agent,
    expected_output=f'A summary of financial data and market position for {companies_str} based on data up to {current_time}.'
)

analysis_task = Task(
    description=f'Analyze the investment potential of {companies_str} using current data up to {current_time}.. And provide the following ratios:\
    - Working Capital Ratio,\
    - Quick Ratio,\
    - Earnings Per Share (EPS),\
    - Price-to-Earnings (P/E) Ratio,\
    - Debt-to-Equity (D/E) Ratio,\
    - Return on Equity (ROE)',
    agent=analysis_agent,
    expected_output=f'Detailed investment analysis for {companies_str} with risk levels based on trends up to {current_time}.'
)

risk_analysis_task = Task(
    description=f'Analyze the risk assessment of {companies_str} using current data up to {current_time}.. And provide the following ratios:\
    - Debt-to-Capital Ratio,\
    - Debt-to-Equity (D/E) Ratio,\
    - Interest Coverage Ratio,\
    - Degree of Combined Leverage',
    agent=risk_analysis_agent,
    expected_output=f'Detailed investment analysis for {companies_str} with risk levels based on trends up to {current_time}.'
)

report_task = Task(
    description=f'Create an investment recommendation report for {companies_str} based on the latest analysis up to {current_time}.',
    agent=report_agent,
    expected_output=f'A structured report with combined analysis and risk assessment to come out with investment recommendations for {companies_str}\
        based on data up to {current_time}. The investment strategy should cover short term and long term strategy. Any additional metrics should be included in the report.',
)

In [40]:
# Create and Run the Crew
investment_crew = Crew(
    agents=[research_agent, analysis_agent, risk_analysis_agent, report_agent],
    tasks=[research_task, analysis_task, risk_analysis_task, report_task],
    verbose=True
)

In [41]:
# Execute the crew
result = investment_crew.kickoff()

# Display and Save Results
print(f"Investment Report for {companies_str} ({filename_datetime}):")
print(result)

# Save with company-specific filename
filename = f"investment_report_{'_'.join(companies)}_{filename_datetime.replace('/', '_')}.txt"
with open(filename, 'w') as f:
    f.write(str(result))
print(f"Report saved to '{filename}'")

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 638f0bc4-6bb9-4de7-bceb-1c7920dea7de                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: e6cd4ef6-8a46-44fe-842e-51f43ba93920
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: e6cd4ef6-8a46-44fe-842e-51f43ba93920
       Status: Executing Task...
    └── 🤖 Agent: Company Researcher
            Status: In Progress

# Agent: Company Researcher
## Task: Research current financial performance, market trends, and news for dbs singapore up to 2025-04-22 15:40:52.684517 using web search.


🚀 Crew: crew
└── 📋 Task: e6cd4ef6-8a46-44fe-842e-51f43ba93920
       Status: Executing Task...
    └── 🤖 Agent: Company Researcher
            Status: In Progress
        └── 🧠 Thinking...

...

🚀 Crew: crew
├── 📋 Task: e6cd4ef6-8a46-44fe-842e-51f43ba93920
│      Assigned to: Company Researcher
│      Status: ✅ Completed
│   └── 🤖 Agent: Company Researcher
│           Status: ✅ Completed
│       └── 🔧 Used Search the internet with Serper (17)
├── 📋 Task: 2b6882e1-a6eb-4b84-a02d-91040d670dea
│      Assigned to: Company Analyst
│      Status: ✅ Completed
│   └── 🤖 Agent: Company Analyst
│           Status: ✅ Completed
│       └── 🔧 Used Search the internet with Serper (19)
├── 📋 Task: 65d13721-cc19-4fd6-88f9-8447062680e1
│      Assigned to: Risk Analyst
│      Status: ✅ Completed
│   └── 🤖 Agent: Risk Analyst
│           Status: ✅ Completed
│       └── 🔧 Used Search the internet with Serper (20)
└── 📋 Task: c956306a-0a84-454c-87a7-bd5fb13f9bfd
       Status: Executing Task...
    └── 🤖 Agent: Investment Advisor
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: e6cd4ef6-8a46-44fe-842e-51f43ba93920
│      Assigned to: Company Researcher
│      Status: ✅ Completed
│   └── 🤖 Agent: Company Researcher
│           Status: ✅ Completed
│       └── 🔧 Used Search the internet with Serper (17)
├── 📋 Task: 2b6882e1-a6eb-4b84-a02d-91040d670dea
│      Assigned to: Company Analyst
│      Status: ✅ Completed
│   └── 🤖 Agent: Company Analyst
│           Status: ✅ Completed
│       └── 🔧 Used Search the internet with Serper (19)
├── 📋 Task: 65d13721-cc19-4fd6-88f9-8447062680e1
│      Assigned to: Risk Analyst
│      Status: ✅ Completed
│   └── 🤖 Agent: Risk Analyst
│           Status: ✅ Completed
│       └── 🔧 Used Search the internet with Serper (20)
└── 📋 Task: c956306a-0a84-454c-87a7-bd5fb13f9bfd
       Assigned to: Investment Advisor
       Status: ✅ Completed
    └── 🤖 Agent: Investment Advisor
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: c956306a-0a84-454c-87a7-bd5fb13f9bfd                                                                     │
│  Agent: Investment Advisor                                                                                      │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 638f0bc4-6bb9-4de7-bceb-1c7920dea7de                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Investment Report for dbs singapore (2025/04/22/15):
Based on the latest analysis and insights available up to April 2025, here is the investment recommendation report for DBS Singapore:

1. **Financial Performance and Outlook**: DBS is projected to maintain a robust financial performance into 2025, with group net interest income expected to slightly surpass the S$15.04 billion achieved in the previous year. This is a positive indication of the bank's ability to generate income, even surpassing its initial forecasts of maintaining 2024 levels. The bank has also reported a record annual net profit of SGD 11.4 billion up by 11%, with a sustained return on equity of 18.0%. 

2. **Share Performance**: The bank's shares reached a record high following an announcement of an improvement in net interest income for 2025. A 4% surge in stock price post-Q4 earnings report reflects strong investor confidence, likely attributed to anticipated continued profit growth and a dividend hike announcement